In [358]:
!pip install pandas
!pip install pickle
!pip install bs4
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Data Analysis Project.

In [359]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
from string import ascii_uppercase as alphabet
import pickle
from scipy.stats import poisson

In [360]:
all_tables = pd.read_html("https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")

# **Extracting world cup data from 1930 to 2018 and 2022 then converting to a data frame an storing in a csv file.**

In [361]:
years = [1930,1934,1938,1942,1946,1950,1954,1958,1962,
         1966,1970,1974,1978,1982,1986,1990,1994,1998,
         2002,2006,2010,2014,2018]

def get_matches(year):
    web = f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"

    response = requests.get(web)
    content = response.text

    soup= BeautifulSoup(content,"lxml")


    matches = soup.find_all("div",class_ = "footballbox")

    home = []
    score =[]
    away =[]
    for match in matches:
        home.append(match.find("th",class_="fhome").get_text())
        score.append(match.find("th",class_="fscore").get_text())
        away.append(match.find("th",class_="faway").get_text())

    dict_football = {"home": home,"score": score,"away":away}

    df_football = pd.DataFrame(dict_football)
    df_football["Year"] = year
    return df_football

  

In [362]:
fifa = [get_matches(year) for year in years]

df_fifa =pd.concat(fifa,ignore_index=True)

df_fifa.to_csv("fifa_world_cup_historical_data.csv",index=False)

df_2022 = get_matches(2022)
df_2022.to_csv("2022 world cup.csv")

In [363]:
historical_data = pd.read_csv("fifa_world_cup_historical_data.csv")
historical_data_extra = pd.read_csv("fifa_world_cup_historical_data.csv")
current_data = pd.read_csv("2022 world cup.csv")
fixture = pd.read_csv("2022 world cup.csv")

# Deleting the Walk over Match in 1938

# **Sweden VS Austria**

In [364]:

delete_index = historical_data[historical_data["home"].str.contains("Sweden") & historical_data["away"].str.contains("Austria")].index
historical_data.drop(index=delete_index,inplace=True)


In [365]:
historical_data[historical_data["home"].str.contains("Sweden") & historical_data["away"].str.contains("Austria")]

,home,score,away,Year


**Removing (a.e.t) from the score. (Referring to score after extra time)**

In [366]:
historical_data["score"] = historical_data["score"].str.replace("[^\d–]","",regex=True)
historical_data["score"]

0      4–1
1      1–0
2      3–0
3      1–0
4      6–3
      ... 
896    2–2
897    1–0
898    2–1
899    2–0
900    4–2
Name: score, Length: 900, dtype: object

In [367]:
historical_data["home"] = historical_data["home"].str.strip()
historical_data["away"] = historical_data["away"].str.strip()

Cleaning  Data br adding home goals and away goals from score – 

In [368]:
historical_data[["Home Goals" , "Away Goals"]] = historical_data["score"].str.split("–",expand=True)
historical_data.drop("score",axis=1,inplace=True)

Rename & change data type

In [369]:
historical_data.rename(columns={"home":"Home","away":"Away","year":"Year"},inplace=True)
historical_data['Home Goals'] = historical_data['Home Goals'].fillna(0)
historical_data['Away Goals'] = historical_data['Away Goals'].fillna(0)
historical_data['Year'] = historical_data['Year'].fillna(0)

historical_data = historical_data.astype({"Home Goals": int, "Away Goals": int, "Year": int})



In [370]:
historical_data["Total Goals"] = historical_data['Home Goals'] +historical_data['Away Goals']


In [371]:
historical_data.to_csv("Cleaned_fifa_world_cup.csv",index=False)

In [372]:
total = 0
home_goals=0
away_goals=0
for year in years:
  matches = historical_data[historical_data['Year'] == year]
  total+=matches["Total Goals"].sum()
  home_goals+=matches["Home Goals"].sum()
  away_goals+=matches["Away Goals"].sum()

In [373]:
cleaned_data = pd.read_csv("Cleaned_fifa_world_cup.csv")

In [374]:
df_home = cleaned_data[["Home","Home Goals","Away Goals"]]
df_away = cleaned_data[["Away","Home Goals","Away Goals"]]
df_home

,Home,Home Goals,Away Goals
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
895,Russia,2,2
896,France,1,0
897,Croatia,2,1
898,Belgium,2,0


In [375]:
df_home = df_home.rename(columns={"Home": "Team", "Home Goals": "GoalsScored", "Away Goals": "GoalsConceded"})
df_away = df_away.rename(columns={"Away": "Team", "Home Goals": "GoalsConceded", "Away Goals": "GoalsScored"})

In [376]:
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby("Team").mean()
df_team_strength.at["Portugal","GoalsScored"]

1.6333333333333333

In [377]:
def predict_points (home, away):
     if home in df_team_strength.index and away in df_team_strength.index:
         # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home, 'GoalsScored']*df_team_strength.at[away, "GoalsConceded"]
        lamb_away = df_team_strength.at[away, 'GoalsScored']*df_team_strength.at[home, "GoalsConceded"]
                                                                
        prob_home, prob_away, prob_draw = 0, 0,0
        for x in range(0,11): #number of goals home 0 team
             for y in range(0, 11): #number of goals away team
                 p = poisson.pmf(x, lamb_home) * poisson. pmf(y, lamb_away)
                 if x==y:
                     prob_draw += p
                 elif x > y:
                     prob_home += p
                 else:
                     prob_away += p
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home,points_away)
     else:
        return(0,0)
 

In [378]:
predict_points("France","Belgium")


(1.987893944001745, 0.8364268216032746)

In [379]:
fixture.loc[:, 'score'] = 0
fixture.drop("Unnamed: 0",axis=1,inplace=True)
group_48 = fixture[:48].copy()
group_knockout = fixture[49:56].copy()
group_knockout.drop("score",axis=1,inplace=True)

group_quarter = fixture[56:60].copy()
group_quarter.drop("score",axis=1,inplace=True)

group_semi = fixture[60:62].copy()
group_final= fixture[62:].copy()
group_48


,home,score,away,Year
0,Qatar,0,Ecuador,2022
1,Senegal,0,Netherlands,2022
2,Qatar,0,Senegal,2022
3,Netherlands,0,Ecuador,2022
4,Ecuador,0,Senegal,2022
5,Netherlands,0,Qatar,2022
6,England,0,Iran,2022
7,United States,0,Wales,2022
8,Wales,0,Iran,2022
9,England,0,United States,2022


In [380]:
dict_table = {}
dict_table_correct = {}
for letter, i in zip(alphabet,range(9,59,7)):
  df = all_tables[i]
  df.rename(columns={df.columns[1]: "Team"},inplace=True)
  df.pop("Qualification")
  dict_table_correct[f"Group {letter}"] = df
  dict_table[f"Group {letter}"] = df
  

In [381]:
t = ["A","B","C","D","E","F","G","H"]
for i in t:
  dict_table[f"Group {i}"].iloc[:, 4:] = 0
  dict_table[f"Group {i}"].loc[:, ["Pld", "W"]] = 0


In [382]:

for group in dict_table:
  teams_in_group = dict_table[group]['Team'].values
  df_fixture_group_6 = group_48[group_48["home"].isin(teams_in_group)]
  for index, row in df_fixture_group_6.iterrows():
    home, away = row['home'], row['away']
    points_home, points_away = predict_points(home, away)
    dict_table[group].loc[dict_table[group]['Team'] ==home, 'Pts'] += points_home
    dict_table[group].loc[dict_table[group]['Team'] ==away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][["Team","pts"]]
    dict_table[group] = dict_table[group].round(0)

group_A = all_tables[9]
group_B = all_tables[16]
group_C = all_tables[23]
group_D = all_tables[30]
group_E = all_tables[37]
group_F = all_tables[44]
group_G = all_tables[51]
group_H = all_tables[58]


In [383]:
dict_table_correct["Group A"]


,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Netherlands,0,0,0,0,0,0,0,0
1,2,Senegal,0,0,0,0,0,0,0,0
2,3,Ecuador,0,0,0,0,0,0,0,0
3,4,Qatar (H),0,0,0,0,0,0,0,0


In [384]:
for group in dict_table:
  winner = dict_table[group].loc[0,"Team"]
  runner_up = dict_table[group].loc[1,"Team"]

  group_knockout.replace({f"Winners {group}":winner,f"Runners-up {group}": runner_up},inplace =True)
group_knockout["Winner"] ="?"
group_knockout

,home,score,away,Year,Winner
49,Argentina,0,Australia,2022,?
50,France,0,Poland,2022,?
51,England,0,Senegal,2022,?
52,Japan,0,Croatia,2022,?
53,Brazil,0,South Korea,2022,?
54,Morocco,0,Spain,2022,?
55,Portugal,0,Switzerland,2022,?


In [385]:
# create get_winner function
def get_winner(df_fixture_updated):
     for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated. loc [index, 'winner'] = winner
     return df_fixture_updated


In [386]:
get_winner(group_knockout)
group_knockout

,home,score,away,Year,Winner,winner
49,Argentina,0,Australia,2022,?,Australia
50,France,0,Poland,2022,?,Poland
51,England,0,Senegal,2022,?,Senegal
52,Japan,0,Croatia,2022,?,Croatia
53,Brazil,0,South Korea,2022,?,South Korea
54,Morocco,0,Spain,2022,?,Spain
55,Portugal,0,Switzerland,2022,?,Switzerland


In [387]:
# create update_table function
def update_table(df_fixture_round_1, df_fixture_round_2):
     for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1. loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
     df_fixture_round_2['winner'] = '?'
     return df_fixture_round_2


In [388]:
update_table(group_knockout,group_quarter)

,home,score,away,Year,winner
56,Croatia,0,Brazil,2022,?
57,Netherlands,0,Argentina,2022,?
58,Morocco,0,Portugal,2022,?
59,England,0,France,2022,?


In [391]:
get_winner(group_quarter)

,home,score,away,Year,winner
56,Croatia,0,Brazil,2022,Brazil
57,Netherlands,0,Argentina,2022,Argentina
58,Morocco,0,Portugal,2022,Portugal
59,England,0,France,2022,France


In [392]:
update_table(group_quarter,group_semi)

,home,score,away,Year,winner
60,Argentina,0,Croatia,2022,?
61,France,0,Morocco,2022,?


In [393]:
get_winner(group_semi)

,home,score,away,Year,winner
60,Argentina,0,Croatia,2022,Croatia
61,France,0,Morocco,2022,Morocco


In [394]:
update_table(group_semi,group_final)

,home,score,away,Year,winner
62,Croatia,0,Morocco,2022,?
63,Argentina,0,France,2022,?


In [395]:
get_winner(group_final)

,home,score,away,Year,winner
62,Croatia,0,Morocco,2022,Morocco
63,Argentina,0,France,2022,France


In [389]:
with open("dict_table","wb") as output:
  pickle.dump(dict_table,output)